<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/fraud/iclaim_ocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip -q install easyocr gradio pythainlp pytesseract opencv-python-headless pdf2image ghostscript
!apt-get install tesseract-ocr tesseract-ocr-th
!sudo apt-get install ghostscript

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package tesseract-ocr-th
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ghostscript is already the newest version (9.55.0~dfsg1-0ubuntu5.9).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [20]:
import os
import re
import cv2
import easyocr
import pytesseract
from pdf2image import convert_from_path
from pythainlp import word_tokenize
from pythainlp.tag import NER
import gradio as gr
from PIL import Image

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
!pip -q install pythainlp PyPDF2 pdfplumber deepcut thaispellcheck oskut sefr_cut transformers torch khanaa pymupdf pytesseract pdf2image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.1/312.1 kB 20.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.6/235.6 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━

In [27]:
from PyPDF2 import PdfReader
from pythainlp.tokenize import word_tokenize
from pythainlp.spell import spell
import re
from pythainlp.spell import correct
import thaispellcheck
import sys
import oskut
import sefr_cut
import numpy as np
import thaispellcheck
import pdfplumber
import fitz

In [29]:
import pdfplumber

# Load the PDF and extract text using pdfplumber
with pdfplumber.open("/content/drive/MyDrive/AIEngineer/fraud/ocr-document/ข้อมูลผู้ป่วยนอก 06a6bf5d-09ce-4f51-aa83-7860621733e4.pdf") as pdf:
    all_text = ""  # Initialize an empty string to store all text
    for page in pdf.pages:  # Iterate through all pages
        text = page.extract_text()  # Extract text from the current page
        all_text += text  # Concatenate the extracted text

print(all_text)  # Print the combined text from all pages

ข้อมูลผู้ป่วยนอก
ข้อมูลส่วนตัว
เลขบัตรประชาชน : 5800100075111
ชื่อ - นามสกุล : นาย จารุณี สุขเกษม รหัสผู้ป่วย (HN) : test12345667
ข้อมูลกรมธรรม์
หมายเลขกรมธรรม์ : GT-000834-00-005456-00 หมายเลขธุรกรรม : 06a6bf5d-09ce-4f51-aa83-7860621733e4
บริษัทประกัน : พรูเด็นเชียลประกันชีวิต เลขที่อ้างอิงจากบริษัทประกัน : ICLM202411046573
ข้อมูลส่งเคลม
ประเภทการรักษา : เจ็บป่วย ประเภทการเข้ารักษา : เข้ารักษาครั้งแรก
อาการสำคัญที่เข้ามาโรงพยาบาล : เจ็บคอ ไปถึงหู
ข้อวินิจฉัยโรค : Acute hepatitis B with delta-agent (coinfection) with hepatic coma
การเจ็บป่วยนี้เกี่ยวข้องกับสิ่งแวดล้อมอื่น
ไม่เกี่ยวข้องกับปัจจัยสิ่งแวดล้อมอื่นๆ
Diagnosis Type Diagnosis (ICD10)
1 โรคหลักที่ให้การรักษา (PP) B160 - ตับอักเสบ บี แบบเฉียบพลัน มีเดลดา-เอเจนต์ (ติดเชื้อร่วมด้วย) และมีภาวะโคม่าจากตับ
(Acute hepatitis B with delta-agent (coinfection) with hepatic coma)
การทำหัตถการ/ผ่าตัด
ไม่ทำหัตถการ
สัญญาณชีพจร
รายการที่ 1
อุณหภูมิในร่างกาย (°C ) ความดันโลหิตค่าบน (mmHg) ความดันโลหิตค่าล่าง (mmHg) ชีพจร (bpm)
75.0 125 80 120
ค

In [34]:
import pdfplumber
import re
import gradio as gr

def extract_text_from_pdf(pdf_path):
    all_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                all_text += text
    return all_text

def parse_patient_info(text):
    patient_info = {}

    patient_info['personal_info'] = re.search(r'ข้อมูลผู้ป่วยนอก\s*ข้อมูลส่วนตัว\s*(.*?)ข้อมูลกรมธรรม์', text, re.DOTALL).group(1).strip()
    patient_info['insurance_info'] = re.search(r'ข้อมูลกรมธรรม์\s*(.*?)ข้อมูลส่งเคลม', text, re.DOTALL).group(1).strip()
    patient_info['claim_info'] = re.search(r'ข้อมูลส่งเคลม\s*(.*?)ข้อวินิจฉัยโรค', text, re.DOTALL).group(1).strip()
    patient_info['diagnosis'] = re.search(r'ข้อวินิจฉัยโรค\s*(.*?)การทำหัตถการ/ผ่าตัด', text, re.DOTALL).group(1).strip()
    patient_info['surgery'] = re.search(r'การทำหัตถการ/ผ่าตัด\s*(.*?)สัญญาณชีพจร', text, re.DOTALL).group(1).strip()
    patient_info['vital_signs'] = re.search(r'สัญญาณชีพจร\s*(.*?)แพทย์ผู้ดูแล', text, re.DOTALL).group(1).strip()
    patient_info['doctor_info'] = re.search(r'แพทย์ผู้ดูแล\s*(.*?)เอกสาร', text, re.DOTALL).group(1).strip()
    patient_info['documents'] = re.search(r'เอกสาร\s*(.*?)รายการค่ารักษาพยาบาล', text, re.DOTALL).group(1).strip()
    patient_info['medical_costs'] = re.search(r'รายการค่ารักษาพยาบาล\s*(.*?)รวมค่ารักษาพยาบาลทั้งหมด', text, re.DOTALL).group(1).strip()

    return patient_info

def process_pdf(file):
    text = extract_text_from_pdf(file.name)
    patient_info = parse_patient_info(text)

    return (
        patient_info['personal_info'],
        patient_info['insurance_info'],
        patient_info['claim_info'],
        patient_info['diagnosis'],
        patient_info['surgery'],
        patient_info['vital_signs'],
        patient_info['doctor_info'],
        patient_info['documents'],
        patient_info['medical_costs']
    )

# สร้างอินเทอร์เฟซ Gradio โดยมีช่องข้อมูลครบถ้วนตามที่ต้องการ
iface = gr.Interface(
    fn=process_pdf,
    inputs=gr.File(label="อัปโหลดไฟล์ PDF"),
    outputs=[
        gr.Textbox(label="ข้อมูลส่วนตัว", lines=2, placeholder="แสดงข้อมูลส่วนตัวที่ดึงมา"),
        gr.Textbox(label="ข้อมูลกรมธรรม์", lines=2, placeholder="แสดงข้อมูลกรมธรรม์ที่ดึงมา"),
        gr.Textbox(label="ข้อมูลส่งเคลม", lines=2, placeholder="แสดงข้อมูลส่งเคลมที่ดึงมา"),
        gr.Textbox(label="ข้อวินิจฉัยโรค", lines=2, placeholder="แสดงข้อวินิจฉัยโรคที่ดึงมา"),
        gr.Textbox(label="การทำหัตถการ/ผ่าตัด", lines=2, placeholder="แสดงการทำหัตถการ/ผ่าตัดที่ดึงมา"),
        gr.Textbox(label="สัญญาณชีพจร", lines=2, placeholder="แสดงสัญญาณชีพจรที่ดึงมา"),
        gr.Textbox(label="แพทย์ผู้ดูแล", lines=2, placeholder="แสดงข้อมูลแพทย์ที่ดึงมา"),
        gr.Textbox(label="เอกสาร", lines=2, placeholder="แสดงเอกสารที่ดึงมา"),
        gr.Textbox(label="รายการค่ารักษาพยาบาล", lines=2, placeholder="แสดงรายการค่ารักษาพยาบาลที่ดึงมา")
    ],
    title="ระบบ OCR สำหรับข้อมูลผู้ป่วย",
    description="อัปโหลดเอกสาร PDF และระบบจะ OCR เพื่อดึงข้อมูลที่ต้องการ"
)

iface.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://03958d338a387d97d4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://03958d338a387d97d4.gradio.live


In [41]:
!pip -q install pdfplumber pytesseract Pillow gradio

In [ ]:
import pdfplumber
import re
import gradio as gr
import pytesseract
from PIL import Image

def extract_text_from_pdf(pdf_path):
    all_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                all_text += text
    return all_text

def extract_text_from_image(image):
    text = pytesseract.image_to_string(image, lang='tha')  # Use pytesseract to extract text from the image
    return text

def parse_patient_info(text):
    patient_info = {}

    patient_info['personal_info'] = re.search(r'ข้อมูลผู้ป่วยนอก\s*ข้อมูลส่วนตัว\s*(.*?)ข้อมูลกรมธรรม์', text, re.DOTALL).group(1).strip()
    patient_info['insurance_info'] = re.search(r'ข้อมูลกรมธรรม์\s*(.*?)ข้อมูลส่งเคลม', text, re.DOTALL).group(1).strip()
    patient_info['claim_info'] = re.search(r'ข้อมูลส่งเคลม\s*(.*?)ข้อวินิจฉัยโรค', text, re.DOTALL).group(1).strip()
    patient_info['diagnosis'] = re.search(r'ข้อวินิจฉัยโรค\s*(.*?)การทำหัตถการ/ผ่าตัด', text, re.DOTALL).group(1).strip()
    patient_info['surgery'] = re.search(r'การทำหัตถการ/ผ่าตัด\s*(.*?)สัญญาณชีพจร', text, re.DOTALL).group(1).strip()
    patient_info['vital_signs'] = re.search(r'สัญญาณชีพจร\s*(.*?)แพทย์ผู้ดูแล', text, re.DOTALL).group(1).strip()
    patient_info['doctor_info'] = re.search(r'แพทย์ผู้ดูแล\s*(.*?)เอกสาร', text, re.DOTALL).group(1).strip()
    patient_info['documents'] = re.search(r'เอกสาร\s*(.*?)รายการค่ารักษาพยาบาล', text, re.DOTALL).group(1).strip()
    patient_info['medical_costs'] = re.search(r'รายการค่ารักษาพยาบาล\s*(.*?)รวมค่ารักษาพยาบาลทั้งหมด', text, re.DOTALL).group(1).strip()

    return patient_info

def process_input(input_data, method):
    if method == "อัปโหลดไฟล์ PDF":  # If it's a PDF file
        text = extract_text_from_pdf(input_data.name)
    else:  # If it's an image scanned
        text = extract_text_from_image(input_data)

    patient_info = parse_patient_info(text)

    return (
        patient_info['personal_info'],
        patient_info['insurance_info'],
        patient_info['claim_info'],
        patient_info['diagnosis'],
        patient_info['surgery'],
        patient_info['vital_signs'],
        patient_info['doctor_info'],
        patient_info['documents'],
        patient_info['medical_costs']
    )

# Create Gradio interface with the required fields
with gr.Blocks() as iface:
    with gr.Row():
        pdf_input = gr.File(label="อัปโหลดไฟล์ PDF", type="filepath")
        image_input = gr.Image(label="สแกนเอกสาร", type="pil", sources="webcam")

    # Output textboxes
    output_boxes = [
        gr.Textbox(label="ข้อมูลส่วนตัว", lines=2, placeholder="แสดงข้อมูลส่วนตัวที่ดึงมา"),
        gr.Textbox(label="ข้อมูลกรมธรรม์", lines=2, placeholder="แสดงข้อมูลกรมธรรม์ที่ดึงมา"),
        gr.Textbox(label="ข้อมูลส่งเคลม", lines=2, placeholder="แสดงข้อมูลส่งเคลมที่ดึงมา"),
        gr.Textbox(label="ข้อวินิจฉัยโรค", lines=2, placeholder="แสดงข้อวินิจฉัยโรคที่ดึงมา"),
        gr.Textbox(label="การทำหัตถการ/ผ่าตัด", lines=2, placeholder="แสดงการทำหัตถการ/ผ่าตัดที่ดึงมา"),
        gr.Textbox(label="สัญญาณชีพจร", lines=2, placeholder="แสดงสัญญาณชีพจรที่ดึงมา"),
        gr.Textbox(label="แพทย์ผู้ดูแล", lines=2, placeholder="แสดงข้อมูลแพทย์ที่ดึงมา"),
        gr.Textbox(label="เอกสาร", lines=2, placeholder="แสดงเอกสารที่ดึงมา"),
        gr.Textbox(label="รายการค่ารักษาพยาบาล", lines=2, placeholder="แสดงรายการค่ารักษาพยาบาลที่ดึงมา")
    ]

    submit_btn = gr.Button("ประมวลผล")

    # Process the input based on which input is provided
    submit_btn.click(process_input, inputs=[pdf_input, gr.Textbox(visible=False)], outputs=output_boxes)  # Pass a dummy input for pdf_input
    submit_btn.click(process_input, inputs=[image_input, gr.Textbox(value="สแกนเอกสาร", visible=False)], outputs=output_boxes)  # Pass a dummy input for image_input

iface.launch(debug=True)
